# Reglas de Asociación: arulesViz R-package

**Santander Meteorology Group**

http://meteo.unican.es

11 Nov 2021


## Resumen

Las librerías [arules](https://cran.r-project.org/web/packages/arules/arules.pdf) y [arulesViz](https://cran.r-project.org/web/packages/arulesViz/arulesViz.pdf) (Hashler 2017) desarrollada por [IDA@SMU](http://lyle.smu.edu/IDA/arules/) consideran diferentes técnicas para la obtención y visualización de reglas de asociación a partir de un conjunto de datos, así como herramientas para explorar dichas reglas.

El objetivo de la presente tarea es familiarizar a los alumnos con el manejo de estas librerías en el marco de R, así como presentar este tipo de algoritmos de aprendizaje el cual, a pesar de no profundizar en el presente curso en él, presenta diversas aplicaciones y nos servirá de introducción a los modelos probabilísticos que analizaremos posteriormente en mayor profundidad.

## Reglas de asociacion en el contexto del aprendizaje estadístico

Las reglas de asociación son uno de los primeros métodos que aparecieron en el campo que hoy se conoce como la _minería de datos_, introducidos por _Agrawal et al._ (1993).

Sea $I = \{i_{1}, i_{2}, \ldots i_{n}\}$ un conjunto de $n$ variables binarias denominadas _items_. Por otra parte, sea $\mathcal{D} = \{ t_{1}, t_{2}, \ldots , t_{n} \}$ un conjunto de transacciones que constituye una _base de datos_. Cada transacción en $\mathcal{D}$ tiene un identificador (ID) único y contiene un subconjunto de elementos en $I$. A partir de esto, una _regla_ se define como una implicación de la forma $X \Rightarrow Y$ donde $X, Y \subseteq I$, y $X \cap Y = \emptyset$. Los conjuntos de _items_ $X$ e $Y$ se denominan _antecedentes_ (a la izquierda, o `LHS` como los veremos a continuación) y _consecuentes_ (a la derecha, `RHS`) de la regla. A menudo las reglas se restringen a un único elemento en el _consecuente_.

Las reglas de asociación son reglas que superan unos umbrales mínimos especificados por el usuario en cuanto a _relevancia_ (_support_, $supp(X)$) y _confianza_ (_confidence_, $conf(X \Rightarrow Y)$).

 * La _relevancia_ (_support_) de un conjunto de items X ($supp(x)$) se define como la **frecuencia de transacciones** en los datos que contienen a dicho conjunto de items
 * La _confianza_ (_confidence_) de una regla ($conf(X \Rightarrow Y) = \frac{supp(X \cup Y)}{supp(X)}$).
 
 Por lo tanto, una regla de asociación $X \Rightarrow Y$ determinada ha de satisfacer dos condiciones:
 
 $$supp(X \cup Y) \geq \sigma$$
 
 y
 
 $$conf(X \Rightarrow Y) \geq \delta$$
 
 siendo $\sigma$ y $\delta$ los umbrales mínimos de _relevancia_ y _confianza_ respectivamente.





## Instalación de la librería arulesViz

Dado que la librería está incluida en el CRAN ([The Comprehensive R Archive Network](https://cran.r-project.org/)), el respositorio central de librerías de R, la instalación puede realizarse a través del comando `install.packages`. En caso de tener ya instalado el paquete no hace falta instalarlo de nuevo y basta con activarlo con el comando `library`.

### Nota para instalación vía Conda (usuarios de Jupyter)

La instalación del paquete `arules`, que implementa los algoritmos ECLAT y APRIORI utilizados para la generación de reglas, puede realizarse de manera sencilla a través del [instalador de Conda](https://anaconda.org/conda-forge/r-arules):  

```
conda install -c conda-forge r-arules
```

Posteriormente, será posible instalar el paquete arulesViz desde la propia sesión de R-jupyter con `install.packages`. **Nota**: en algunos casos puede ser necesario instalar algunas dependencias antes de `arulesViz`, tales como el paquete `igraph`


In [ ]:
library("arules")

In [ ]:
# If we have not installed the "arulesViz" library we must do it first:
if (!require(arulesViz)) install.packages("arulesViz", dependencies = TRUE)

Nótese que la instalación de la libería requiere de la instalación de sus dependencias. En particular la librería `arules`, la cual se activa al activar `arulesViz`.

## Ejemplo realizado en clase:

Inicialmente consideraremos el ejemplo realizado en clase para familiarizarnos con los comandos y sus opciones para luego aplicarlo en un problema más general. Para ello, primero definiremos la tabla de transacciones del problema:

### Definición del conjunto de datos de ejemplo y conversión a la clase `transactions`

In [ ]:
itemset <- paste0("item", 1:10)
base::sample(itemset, size = 3, replace = FALSE)

In [ ]:
table <- list(c("p","l","O","b"), c("p","l"), c("p","O","c"), c("p","l","O","c"))
transactions <- as(table, "transactions")
inspect(transactions)

### Inspección del conjunto de transacciones y conjuntos de items

Podemos ver el soporte de cada item. Con el argumento `type` se especifica si la frecuencia se devuelve en términos absolutos o relativos. 

In [ ]:
itemFrequency(transactions, type = "a")
itemFrequency(transactions, type = "r")

El método `itemFrequencyPlot` permite representar gráficamente los datos mediante un diagrama de barras:

In [ ]:
itemFrequencyPlot(transactions)

Entre los argumentos opcionales del gráfico de frecuencias, existe la posibilidad de filtrar por un valor mínimo de *soporte* del set de items, deshechando los valores menores que el indicado. Por ejemplo:

In [ ]:
itemFrequencyPlot(transactions, support = .5)

### Generación de conjuntos de reglas a partir de los datos

Una vez definida la tabla de transacciones, buscaremos los itemsets más frecuentes a través de los dos algoritmos explicados:

#### Algortimo APRIORI:

In [ ]:
# Ejecución del algoritmo 'apriori'
iApriori <- apriori(transactions, parameter = list(supp = 0, conf = 0, target = "frequent itemsets"))

# Inspección del resultado
inspect(iApriori)


y 

#### Algoritmo ECLAT:

In [ ]:
# Ejecución del algoritmo 'eclat'
iEclat <- eclat(transactions, parameter = list(support = 0, maxlen = 5, tidLists = TRUE))

# Inspección del resultado
inspect(iEclat)

### Inspección del conjunto de reglas. Filtrado y redundancia.

Notar que ECLAT suprime aquellos itemsets que no ocurren ($supp(X)=0$) mientras que, si no se establecen restricciones, APRIORI devuelve todas las combinaciones, salvo el conjunto vacío y el total. Podemos filtrar las transacciones según incluyan ciertos productos con el método `subset`. La sintaxis `%in%`, `%ain%`, `%pin%` y `%oin` sirve para filtrar items en el subconjunto de salida.

In [ ]:
# %in% filtra los sets de item que contienen "b" o "l" o ambas
inspect(subset(iApriori, subset = items %in% c("b","l")))


In [ ]:
# %ain% filtra los sets de item que contienen "b" y "l" simultáneamente
inspect(subset(iApriori, subset = items %ain% c("b","l")))


In [ ]:
# %pin% realiza "partial matching", de acuerdo con la expresión indicada.
# Este ejemplo no es el mejor, porque los códigos de los items son un únic caracter.
# Ver ?arules::match para un ejemplo mejor

inspect(subset(iApriori, subset = items %pin% c("b")))

In [ ]:
# %oin% ("only in") es más restrictivo que %in%: 
# Filtra los itemsets que únicamente contienen los elementos indicados:

inspect(subset(iApriori, subset = items %oin% c("b","l")))

Como hemos visto, una vez obtenidos los itemsets de partida, se construyen las reglas de asociación en base a las restricciones que se impongan. En el caso del algoritmo APRIORI la función permite obtener las reglas como salida. Sin embargo, el algoritmo ECLAT únicamente devuelve los itemsets, y las reglas de asociación deben inducirse con la función `ruleInduction`.

In [ ]:
# El parámetro target = "rules" indica al algoritmo apriori la generación de reglas:

rApriori <- apriori(transactions, parameter = list(supp = 0, conf = 0, target = "rules"))
inspect(rApriori)

La inclusión de reglas cuyo antecedente (`lhs`: left hand side) es el conjunto vacío (`{}`) pueden fitrarse incluyendo restricciones respecto a la longitud mínima (`minlen`).

In [ ]:
rApriori <- apriori(transactions, parameter = list(supp = 0, conf = 0, target = "rules", minlen = 2))
inspect(rApriori)

En el caso de ECLAT, como vimos, hay que hacer uso de funciones específicas de obtención de reglas:

In [ ]:
rEclat <- ruleInduction(iEclat, transactions, confidence = 0, control = list(method = "ptree", reduce = TRUE))
inspect(rEclat)

Al filtrar itemsets con soporte nulo e incluir la reducción en el algoritmo notar que el número de reglas que aparecen en el algoritmo ECLAT es sensiblemente inferior al obtenido en el algoritmo APRIORI.

Al inspeccionar el objeto, bien con las reglas bien con los itemsets, se muestran por defecto tres parámetros, support, confidence y lift, si bien se pueden obtener un gran número de medidas de interés alternativas, gran parte de las cuales pueden usarse también en el proceso de aprendizaje:

In [ ]:
interestMeasure(rEclat, c("support", "chiSquare", "confidence", "conviction","cosine", "coverage", "leverage", "lift", "oddsRatio"), transactions)

Si bien no hemos realizado ningún filtro a las reglas obtenidas, más allá de las impuestas por los algoritmos de aprendizaje, es habitual obtener reglas redundantes la cuales debemos filtrar. Para ello, la función `is.redundant` nos permite localizar estas redundancias:

In [ ]:
indRedundantApriori <- which(is.redundant(rApriori))
inspect(rApriori[indRedundantApriori])

Procedemos a filtrar dichas redes:

In [ ]:
rAprioriFiltered <- rApriori[!is.redundant(rApriori)]
rEclatFiltered <- rEclat[!is.redundant(rEclat)]

e inspeccionamos de nuevo las reglas obtenidas con el algoritmo APRIORI

In [ ]:
inspect(rAprioriFiltered)

y con el algoritmo ECLAT

In [ ]:
rEclatFiltered <- rEclat[!is.redundant(rEclat)]
inspect(rEclatFiltered)

Finalmente, podemos resolver dudas como las siguientes: 
* Encontrar todas las reglas relacionando cualquier producto con uno dado (p.e. O - Naranja).

In [ ]:
inspect(subset(rAprioriFiltered, subset = rhs %in% c("O")))
inspect(subset(rEclatFiltered, subset = rhs %in% c("O")))

* Encontrar todas las reglas que dependen de un producto dado (p.e. O - Naranja).

In [ ]:
inspect(subset(rAprioriFiltered, subset = lhs %in% c("O")))
inspect(subset(rEclatFiltered, subset = lhs %in% c("O")))

* Encontrar todas las reglas que cumplan los dos criterios anteriores para ciertos productos dados.

In [ ]:
inspect(subset(rAprioriFiltered, subset = rhs %in% c("O") | lhs %in% c("O")))
inspect(subset(rEclatFiltered, subset = rhs %in% c("O") | lhs %in% c("O")))

* Encontrar el conjunto de reglas con mayor confianza cumpliendo alguno de los criterios anteriores.

In [ ]:
inspect(subset(rAprioriFiltered, subset = rhs %in% c("O") | lhs %in% c("O") & confidence >= 0.5))
inspect(subset(rEclatFiltered, subset = rhs %in% c("O") | lhs %in% c("O") & confidence >= 0.5))

* Consultar el conjunto de reglas con alguno de los parámetros con valores más altos (`head`) o bajos (`tail`)

In [ ]:
inspect(head(sort(rAprioriFiltered, by ="lift"),3))
inspect(head(sort(rEclatFiltered, by ="lift"),3))
inspect(tail(sort(rAprioriFiltered, by ="lift"),3))
inspect(tail(sort(rEclatFiltered, by ="lift"),3))

## Herramientas de visualización

Finalmente, una vez obtenidas las reglas de asociación, podemos visualizar los resultados.

***
**NOTA:** En los casos presentados, la función `plot` aplicada a la clase `transactions` permite la opción de activar el motor gráfico [htmlwidgets de R](https://www.htmlwidgets.org/), siempre que éste se encuentre disponible en nuestro entorno de trabajo, mediante la opción `engine = "htmlwidget"`. En general, la mayor parte de los métodos gráficos de exploración de reglas de asociación son útiles bajo este modo de representación gráfica interactiva, que permite al usuario identificar elementos gráficos (conjuntos de regals, _itemsets_, etc.) de manera interactiva desplazando el puntero, así como funciones de acercar/alejar y encuadre.

***


Para instalar el paquete htmlwidgets uede utilizarse install.packages("htmlwidgets") o bien, en un entorno Conda de Jupyter, mediante la [receta correspondiente](https://anaconda.org/r/r-htmlwidgets):

```
conda install -c r r-htmlwidgets
``` 

Un ejemplo de visualización interactiva desde Jupyter notebook ([ver fuente](https://blog.ouseful.info/2018/04/26/r-htmlwidgets-in-jupyter-notebooks/)):


```r
m <- plot(rEclatFiltered, method = "matrix", measure="lift", engine = "htmlwidget")
htmlwidgets::saveWidget(m, 'demo.html', selfcontained = FALSE)
IRdisplay::display_html('<iframe src="demo.html"></iframe>')
```



### Matriz de itemsets:

In [ ]:
image(iApriori@items, xlab = "Items", ylab = "Itemsets")
image(iEclat@items, xlab = "Items", ylab = "Itemsets")

### Diagramas de dispersión (_scatterplots_) de reglas:

In [ ]:
plot(rAprioriFiltered)

In [ ]:
plot(rEclatFiltered, engine = "htmlwidget")

### Matrices de asociaciones:

In [ ]:
plot(rAprioriFiltered, method = "matrix", measure="lift")

In [ ]:
plot(rEclatFiltered, method = "matrix", measure="lift")


- Matriz de asociaciones por grupos, incluyendo parámetros:

In [ ]:
plot(rAprioriFiltered, method="grouped", measure="lift")
plot(rEclatFiltered, method="grouped", measure="support")


### Enlaces de las reglas:

In [ ]:
plot(rAprioriFiltered, method="paracoord")
plot(rEclatFiltered, method="paracoord")

- Estadística de una regla en particular:

In [ ]:
oneRule <- sample(rAprioriFiltered, 1)
inspect(oneRule)
plot(oneRule, method="doubledecker", data = transactions)

In [ ]:
oneRule <- sample(rEclatFiltered, 1)
inspect(oneRule)
plot(oneRule, method="doubledecker", data = transactions)

### Visualización basada en grafos:
Esta visualización resulta útil en modo interactivo, a través de la opción `engine = "htmlwidgets"` si esta se encuentra disponible en el entorno de trabajo

In [ ]:
plot(rEclatFiltered, method = "graph")

## Base de datos Groceries

En la presente tarea trabajaremos con la base de datos `Groceries`, disponibles públicamente para su uso desde R. Para una breve descripción de esta base de datos puede recurrir a la ayuda de R.

In [ ]:
# Help command in R: ?
? Groceries
# Loading the dataset
data(Groceries)
Groceries@itemInfo$labels

¿Cuantas transacciones y clases contiene el dataset?

In [ ]:
# Incluye el código para responder a la cuestion anterior. ? str

Notar que, para inspeccionar el objeto, hemos usado en un caso el símbolo arroba (`@`) y en otro el dolar (`$`). Esto se debe a que en el primer caso es un objeto tipo S3 ó S4, mientras que en el segundo caso es una colección de datos o `data.frame`.

# Aprendiendo de los datos: Algoritmo A Priori

In [ ]:
? apriori

Las reglas de asociación son implicaciones de la forma `X=>Y`, las cuales se fundamentan en dos conceptos básicos: la confianza y la relevancia. El primero es la proporción de registros con `X` que contienen también `Y`, mientras que el segundo es el número de registros que contienen tanto X como Y. Dada una relevancia mínima `Rmin`, el algoritmo a priori consta de 6 pasos:

 * Definimos `i=1`, que se corresponde con el tamaño de los conjuntos.
 * Generamos un conjunto unitario para cada atributo en `A1`.
 * Comprobamos la relevancia de todos los conjuntos de `Ai`, eliminando aquellos cuya relevancia es menor que el umbral (`relevancia < Rmin`).
 * Combinamos los conjuntos de `Ai` creando conjuntos de tamaño `i+1` en `Ai+1`
 * Si `Ai` no es vacío entonces hacemos `i=i+1` y volvemos al paso 3.
 * Si `Ai` es vacío entonces el conjunto de reglas de asociación será la unión de todos los `Ai` construidos (`A2 U A3 U ... U Ai`).
 
El algoritmo anterior está implementado en la librería `arulesViz` por lo que, a través de la ayuda, podemos ver como usarlo y hacer un ejemplo:

In [ ]:
data(Groceries)
rules <- apriori(Groceries, parameter=list(support = 0.001, confidence = 0.5))

Podemos inspeccionar el objeto y ver algunas de las reglas. Por ejemplo, el siguiente comando nos mostrará las primeras 4 reglas en función de su confianza:

In [ ]:
inspect(head(sort(rules, by = "confidence"),4))

Para ver cómo funcionan los comandos anteriores puedes usar la ayuda de R:
`? sort`
`? inspect`
`? head`

## Herramientas de visualización

En los puntos anteriores hemos establecido las reglas de asociación y manejado la estructura obtenida. Sin embargo, la librería `aruleViz` incluye herramientas de visualización a través de la sobrecarga de la función `plot` (`? plot`).

In [ ]:
plot(rules, method = NULL, measure = "support", shading = "lift", interactive = FALSE, data = NULL, control = NULL)

## Práctica:

Responde a las siguientes cuestiones, incluyendo cuando corresponda el código asociado:

 * ¿Cuantas reglas de asociación se han generado? 

In [ ]:
# Incluye el código para responder a la cuestion anterior.

 * ¿Qué porcentaje de reglas es redundante? 

In [ ]:
# Incluye el código para responder a la cuestion anterior.

 * Teniendo en cuenta el conjunto de datos de partida, ¿resulta útil el conjunto de reglas generado?

In [ ]:
# Incluye el código para responder a la cuestion anterior.

 * ¿Cual es la regla con mayor confianza? ¿Y con menor?

In [ ]:
# Incluye el código para responder a la cuestion anterior.

 * ¿Cual es la regla con mayor interés? ¿Y con menor?

In [ ]:
# Incluye el código para responder a la cuestion anterior.

 * En el objeto aparece una medida llamada `lift`, ¿qué mide?

In [ ]:
# Incluye el código para responder a la cuestion anterior.

 * Escribe el `podium` asociado a cada una de las tres medidas: `support`, `confidence` y `lift`.

In [ ]:
# Incluye el código para responder a la cuestion anterior.

 * ¿Se puede inferir que la regla es persistente respecto a la medida de interés?

In [ ]:
# Incluye el código para responder a la cuestion anterior.

 * Explorar los ejemplos de la función plot. 

In [ ]:
# Incluye el código para responder a la cuestion anterior.

 * ¿Hay alguna agrupación de items que se de con gran frecuencia? Por ejemplo, ¿los productos de limpieza se compran de forma conjunta? ¿Los productos de cuidado personal?, etc...

In [ ]:
# Incluye el código para responder a la cuestion anterior.

 * Responder a las preguntas anteriores con los diferentes algoritmos disponibles, comparar los resultados y concluir si los resultados son o no robustos respecto al algoritmo.

In [ ]:
# Incluye el código para responder a la cuestion anterior.

## Referencias

* Agrawal R, Imielinski T, Swami A (1993). Mining Association Rules between Sets of Items in Large Databases. In Proceedings of the 1993 ACM SIGMOD International Conference on Management of Data, pp. 207–216. ACM Press. URL http://doi.acm.org/10.1145/170035.170072.
* Hahsler M (2017). “arulesViz: Interactive Visualization of Association Rules with R.” _R Journal_, *9*(2),163-175. ISSN 2073-4859, <URL:https://journal.r-project.org/archive/2017/RJ-2017-047/RJ-2017-047.pdf>.

## Session Info:

In [ ]:
print(sessionInfo())